In [2]:
# Import Dependencies
import pandas as pd
import numpy as np

In [3]:
# Read in population data
population_df = pd.read_csv('co_population.csv')
population_df.head()

,county,2010,2011,2012,2013,2014,2015,2016
0,Adams County,441603,451443,459861,469377,479488,489822,497395
1,Alamosa County,15445,16117,16092,16200,16213,16305,16456
2,Arapahoe County,572003,584955,595094,606839,617385,628834,636949
3,Archuleta County,12084,12014,12124,12205,12222,12382,12836
4,Baca County,3788,3778,3722,3655,3586,3552,3548


In [4]:
transpose columns to rows
population_df = population_df.melt(id_vars=["county"], 
        var_name="year", 
        value_name="population")

In [5]:
# Export file to examine
#file_name = ('population_group.xlsx')
#population_df.to_excel(file_name)

In [6]:
# Sort in chronological order
sorted_pop_df = population_df.sort_values(['county', 'year']).reset_index()
sorted_pop_df

,index,county,year,population
0,0,Adams County,2010,441603
1,63,Adams County,2011,451443
2,126,Adams County,2012,459861
3,189,Adams County,2013,469377
4,252,Adams County,2014,479488
...,...,...,...,...
436,188,Yuma County,2012,10076
437,251,Yuma County,2013,10112
438,314,Yuma County,2014,10175
439,377,Yuma County,2015,10064


In [7]:
# Drop extra index column
sorted_pop_df.drop(columns=['index'], inplace=True)
sorted_pop_df.head()

,county,year,population
0,Adams County,2010,441603
1,Adams County,2011,451443
2,Adams County,2012,459861
3,Adams County,2013,469377
4,Adams County,2014,479488


In [8]:
# Remove word "County" from county column
sorted_pop_df['county'] = sorted_pop_df['county'].str.replace(' County', '')

In [9]:
# Check county names for later merge
sorted_pop_df.county.unique()

array(['Adams', 'Alamosa', 'Arapahoe', 'Archuleta', 'Baca', 'Bent',
       'Boulder', 'Broomfield', 'Chaffee', 'Cheyenne', 'Conejos',
       'Costilla', 'Crowley', 'Custer', 'Delta', 'Denver', 'Dolores',
       'Douglas', 'Eagle', 'El Paso', 'Elbert', 'Fremont', 'Garfield',
       'Gilpin', 'Grand', 'Gunnison', 'Hinsdale', 'Huerfano', 'Jackson',
       'Jefferson', 'Kiowa', 'Kit Carson', 'La Plata', 'Lake', 'Larimer',
       'Las Animas', 'Lincoln', 'Logan', 'Mesa', 'Mineral', 'Moffat',
       'Montezuma', 'Montrose', 'Morgan', 'Otero', 'Ouray', 'Park',
       'Phillips', 'Pitkin', 'Prowers', 'Pueblo', 'Rio Blanco',
       'Rio Grande', 'Routt', 'Saguache', 'San Juan', 'San Miguel',
       'Sedgwick', 'Summit', 'Teller', 'Washington', 'Weld', 'Yuma'],
      dtype=object)

In [10]:
# Read in economy data
co_econ_price_df = pd.read_csv('co_econ_price.csv')
co_econ_price_df.head()

,County,Year,GDP_pct_Change,House_price
0,Adams,2010,6.78,"$186,966.95"
1,Adams,2011,15.78,"$176,663.41"
2,Adams,2012,-0.17,"$181,639.65"
3,Adams,2013,10.03,"$204,357.18"
4,Adams,2014,17.60,"$228,642.09"


In [11]:
# Check county names for later merge
co_econ_price_df.County.unique()

array(['Adams', 'Alamosa', 'Arapahoe', 'Archuleta', 'Baca', 'Bent',
       'Boulder', 'Broomfield', 'Chaffee', 'Cheyenne', 'Conejos',
       'Costilla', 'Crowley', 'Custer', 'Delta', 'Denver', 'Dolores',
       'Douglas', 'Eagle', 'Elbert', 'Fremont', 'Garfield', 'Gilpin',
       'Grand', 'Gunnison', 'Hinsdale', 'Huerfano', 'Jackson',
       'Jefferson', 'Kiowa', 'Lake', 'Larimer', 'Lincoln', 'Logan',
       'Mesa', 'Mineral', 'Moffat', 'Montezuma', 'Montrose', 'Morgan',
       'Otero', 'Ouray', 'Park', 'Phillips', 'Pitkin', 'Prowers',
       'Pueblo', 'Routt', 'Saguache', 'Sedgwick', 'Summit', 'Teller',
       'Washington', 'Weld', 'Yuma'], dtype=object)

In [12]:
# Remove non-matching county rows
clean_pop_df = sorted_pop_df[~sorted_pop_df["county"].isin(['El Paso', 'Las Animas', 'Rio Blanco', 'Rio Grande', 'San Juan', 'San Miguel'])]
clean_pop_df

,county,year,population
0,Adams,2010,441603
1,Adams,2011,451443
2,Adams,2012,459861
3,Adams,2013,469377
4,Adams,2014,479488
...,...,...,...
436,Yuma,2012,10076
437,Yuma,2013,10112
438,Yuma,2014,10175
439,Yuma,2015,10064


In [13]:
# Change column names to match
co_econ_price_df.rename(columns={'County': 'county', 'Year':'year'}, inplace=True)

In [14]:
# Check data types for econ
co_econ_price_df.dtypes

county             object
year                int64
GDP_pct_Change    float64
House_price        object
dtype: object

In [15]:
# Check data types for population
clean_pop_df.dtypes

county        object
year          object
population     int64
dtype: object

In [16]:
# Change year to integer
clean_pop_df['year'] = clean_pop_df['year'].astype(int)

In [17]:
# Check data types
clean_pop_df.dtypes

county        object
year           int32
population     int64
dtype: object

In [18]:
# Merge dataframes
new_county_df = pd.DataFrame.merge(clean_pop_df, co_econ_price_df, how='left', left_on=['county', 'year'], right_on = ['county', 'year'])
new_county_df

,county,year,population,GDP_pct_Change,House_price
0,Adams,2010,441603,6.78,"$186,966.95"
1,Adams,2011,451443,15.78,"$176,663.41"
2,Adams,2012,459861,-0.17,"$181,639.65"
3,Adams,2013,469377,10.03,"$204,357.18"
4,Adams,2014,479488,17.60,"$228,642.09"
...,...,...,...,...,...
394,Yuma,2012,10076,-2.76,"$102,651.14"
395,Yuma,2013,10112,2.24,"$110,623.48"
396,Yuma,2014,10175,18.03,"$120,436.30"
397,Yuma,2015,10064,-2.82,"$131,109.81"


In [19]:
# Read in pricing out dataset
pricing_out_df = pd.read_csv('pricing_out.csv')
pricing_out_df.head()

,Unnamed: 0,year,county,Description,GDP_Change,House_price,Income,interest_rate,lending_limit,periods,suggest_payment,suggest_income,priced_out
0,0,2010,Adams,"Agriculture, forestry, fishing and hunting",31.9,186966.95,52785,0.0025,0.25,360,788.260203,37836.48974,N
1,1,2010,Adams,"Mining, quarrying, and oil and gas extraction",1.4,186966.95,52785,0.0025,0.25,360,788.260203,37836.48974,N
2,2,2010,Adams,Manufacturing,-3.5,186966.95,52785,0.0025,0.25,360,788.260203,37836.48974,N
3,3,2010,Adams,"Educational services, health care, and social ...",6.5,186966.95,52785,0.0025,0.25,360,788.260203,37836.48974,N
4,4,2010,Adams,"Arts, entertainment, recreation, accommodation...",2.5,186966.95,52785,0.0025,0.25,360,788.260203,37836.48974,N


In [20]:
# Check counties for merge
pricing_out_df.county.unique()

array(['Adams', 'Alamosa', 'Arapahoe', 'Archuleta', 'Baca', 'Bent',
       'Boulder', 'Broomfield', 'Chaffee', 'Cheyenne', 'Conejos',
       'Costilla', 'Crowley', 'Custer', 'Delta', 'Denver', 'Dolores',
       'Douglas', 'Eagle', 'Elbert', 'Fremont', 'Garfield', 'Gilpin',
       'Gunnison', 'Hinsdale', 'Huerfano', 'Jefferson', 'Kiowa', 'Lake',
       'Larimer', 'Lincoln', 'Logan', 'Mesa', 'Mineral', 'Moffat',
       'Montezuma', 'Montrose', 'Morgan', 'Ouray', 'Park', 'Phillips',
       'Pitkin', 'Prowers', 'Pueblo', 'Routt', 'Sedgwick', 'Summit',
       'Teller', 'Washington', 'Weld', 'Yuma', 'Grand', 'Otero',
       'Saguache', 'Jackson'], dtype=object)

In [21]:
# Counties in alphabetical order
pricing_out_df = pricing_out_df.sort_values(['county'])

In [22]:
# Check counties again
pricing_out_df.county.unique()

array(['Adams', 'Alamosa', 'Arapahoe', 'Archuleta', 'Baca', 'Bent',
       'Boulder', 'Broomfield', 'Chaffee', 'Cheyenne', 'Conejos',
       'Costilla', 'Crowley', 'Custer', 'Delta', 'Denver', 'Dolores',
       'Douglas', 'Eagle', 'Elbert', 'Fremont', 'Garfield', 'Gilpin',
       'Grand', 'Gunnison', 'Hinsdale', 'Huerfano', 'Jackson',
       'Jefferson', 'Kiowa', 'Lake', 'Larimer', 'Lincoln', 'Logan',
       'Mesa', 'Mineral', 'Moffat', 'Montezuma', 'Montrose', 'Morgan',
       'Otero', 'Ouray', 'Park', 'Phillips', 'Pitkin', 'Prowers',
       'Pueblo', 'Routt', 'Saguache', 'Sedgwick', 'Summit', 'Teller',
       'Washington', 'Weld', 'Yuma'], dtype=object)

In [23]:
# Reorder columns to match and leave out unnecessary data
pricing_out_df= pricing_out_df[['county', 'year', 'Income', 'priced_out']]
pricing_out_df

,county,year,Income,priced_out
0,Adams,2010,52785,N
1555,Adams,2015,62991,N
1554,Adams,2015,62991,N
1884,Adams,2016,65442,N
311,Adams,2011,52429,N
...,...,...,...,...
607,Yuma,2011,46246,Y
608,Yuma,2011,46246,Y
609,Yuma,2011,46246,Y
611,Yuma,2011,46246,Y


In [24]:
# Drop duplicate rows
pricing_out_df.drop_duplicates(inplace=True)

In [25]:
pricing_out_df

,county,year,Income,priced_out
0,Adams,2010,52785,N
1555,Adams,2015,62991,N
1884,Adams,2016,65442,N
311,Adams,2011,52429,N
1231,Adams,2014,59316,N
...,...,...,...,...
301,Yuma,2010,43361,Y
914,Yuma,2012,45059,Y
1553,Yuma,2014,46838,Y
610,Yuma,2011,46246,Y


In [26]:
pricing_out_df.reset_index(inplace=True)

In [27]:
pricing_out_df

,index,county,year,Income,priced_out
0,0,Adams,2010,52785,N
1,1555,Adams,2015,62991,N
2,1884,Adams,2016,65442,N
3,311,Adams,2011,52429,N
4,1231,Adams,2014,59316,N
...,...,...,...,...,...
364,301,Yuma,2010,43361,Y
365,914,Yuma,2012,45059,Y
366,1553,Yuma,2014,46838,Y
367,610,Yuma,2011,46246,Y


In [28]:
# Drop extra index
pricing_out_df.drop(columns=['index'], inplace=True)
pricing_out_df

,county,year,Income,priced_out
0,Adams,2010,52785,N
1,Adams,2015,62991,N
2,Adams,2016,65442,N
3,Adams,2011,52429,N
4,Adams,2014,59316,N
...,...,...,...,...
364,Yuma,2010,43361,Y
365,Yuma,2012,45059,Y
366,Yuma,2014,46838,Y
367,Yuma,2011,46246,Y


In [29]:
# Merge 3rd dataset
final_county_df = pd.DataFrame.merge(new_county_df, pricing_out_df, how='left', left_on=['county', 'year'], right_on = ['county', 'year'])
final_county_df

,county,year,population,GDP_pct_Change,House_price,Income,priced_out
0,Adams,2010,441603,6.78,"$186,966.95",52785.0,N
1,Adams,2011,451443,15.78,"$176,663.41",52429.0,N
2,Adams,2012,459861,-0.17,"$181,639.65",55695.0,N
3,Adams,2013,469377,10.03,"$204,357.18",54876.0,N
4,Adams,2014,479488,17.60,"$228,642.09",59316.0,N
...,...,...,...,...,...,...,...
394,Yuma,2012,10076,-2.76,"$102,651.14",45059.0,Y
395,Yuma,2013,10112,2.24,"$110,623.48",45487.0,Y
396,Yuma,2014,10175,18.03,"$120,436.30",46838.0,Y
397,Yuma,2015,10064,-2.82,"$131,109.81",48934.0,Y


In [30]:
final_county_df.shape

(399, 7)

In [33]:
# Change column names
final_county_df.rename(columns={'county': 'County', 'year':'Year', 'House_price': 'House_Price', 'priced_out': 'Priced_Out'}, inplace=True)

In [34]:
# Remove dollar sign from House_Price
final_county_df['House_Price'] = final_county_df['House_Price'].str.replace('$', '')

In [35]:
# Remove trailing spaces
final_county_df.House_Price = final_county_df['House_Price'].str.strip()

In [36]:
# Remove commas
final_county_df['House_Price'] = final_county_df['House_Price'].str.replace(',', '')

In [37]:
# Change House_Price to float
h= final_county_df['House_Price']
final_county_df.House_Price = pd.to_numeric(h)

In [38]:
# Remove decimals
final_county_df.House_Price = final_county_df.House_Price.round()

In [39]:
# Drop null values
final_county_df.dropna(inplace=True)

In [40]:
# Print final dataframe
final_county_df

,County,Year,population,GDP_pct_Change,House_Price,Income,Priced_Out
0,Adams,2010,441603,6.78,186967.0,52785.0,N
1,Adams,2011,451443,15.78,176663.0,52429.0,N
2,Adams,2012,459861,-0.17,181640.0,55695.0,N
3,Adams,2013,469377,10.03,204357.0,54876.0,N
4,Adams,2014,479488,17.60,228642.0,59316.0,N
...,...,...,...,...,...,...,...
394,Yuma,2012,10076,-2.76,102651.0,45059.0,Y
395,Yuma,2013,10112,2.24,110623.0,45487.0,Y
396,Yuma,2014,10175,18.03,120436.0,46838.0,Y
397,Yuma,2015,10064,-2.82,131110.0,48934.0,Y


In [41]:
# export to excel
file_name = 'final_county.xlsx'
final_county_df.to_excel(file_name)